In [139]:
import sys
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,
    )
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AR


class Peggy(object):
    def __init__(self, session, date, tickers, start_time, end_time):
        self.session = session
        self.date = date
        self.tickers = tickers
        self.ticker1 = self.tickers[0]
        self.ticker2 = self.tickers[1]
        self.start_time = start_time
        self.end_time = end_time
        self.interval = string_to_micro("1s")
        self.state = 'NULL'
        self.side1 = 0
        self.side2 = 0
        self.order_size = 1
        self.buy_shares1 = 0
        self.buy_dollars = 0
        self.sell_shares1 = 0
        self.sell_dollars = 0
        self.buy_shares2 = 0
        self.sell_shares2 = 0
        self.open_longs = 0
        self.open_shorts = 0
        self.close_longs = 0
        self.close_shorts = 0
        
        self.runs = 0
        
        self.dt = 1
        self.long_open = 1.25
        self.long_close = 0.50
        self.short_open = 1.25;
        self.short_close = 0.75
        self.training_size = 100
        
        self.session.add_timer(self.start_time, self.start_callback)
        
        self.results = {'time': []}
        for ticker in self.tickers:
            self.results[ticker] = []
            self.results['return {}'.format(ticker)] = []
        
        # subscribe to the ticker of interest, and set the first timer
        for ticker in self.tickers:
            self.session.subscribe_ticker_all_feeds(ticker)
    
    def start_callback(self, time):
        for ticker in self.tickers:
            self.session.subscribe_event(ticker, ORDER_EVENTS, self.event_callback)
        self.session.add_timer(time, self.timer_callback)
        
    
    def event_callback(self, ticker, event_params):
        self.process_executions(event_params)
        
    def timer_callback(self, time):
        # get the best bid and offer, compute the midmarket, and update averages
        #self.process_executions(event_params)
        self.runs += 1
        self.results['time'].append(micro_to_time(time))
        bid1, ask1 = self.session.get_inside_market(self.ticker1)
        bid2, ask2 = self.session.get_inside_market(self.ticker2)
        
        self.results[self.ticker1].append(self.get_midmarket(self.ticker1) / 1000000.0)
        self.results[self.ticker2].append(self.get_midmarket(self.ticker2) / 1000000.0)
        
        if time > self.start_time + 10**6:
            self.results['return {}'.format(self.ticker1)].append(np.float(self.returns(self.results[self.ticker1][-2:])))
            self.results['return {}'.format(self.ticker2)].append(np.float(self.returns(self.results[self.ticker2][-2:])))
        
        if time > self.start_time + self.training_size * 10**6:   
            returns1 = self.results['return {}'.format(self.ticker1)][-100:]
            returns2 = self.results['return {}'.format(self.ticker2)][-100:]
            residuals, a,b = self.regress(returns1,returns2)
            
            kappa, m, sigma, sigmaeq = self.fitOU(residuals)
            
            s = self.sscore(m, sigmaeq)
            pos = self.buy_shares1 - self.sell_shares1
            self.order_size = 1
            
            # add feature to check if we have orders at the market before we open a position
            orders = self.session.get_all_orders()
        
            if not orders:
                if pos == 0:
                    if s < -self.long_open:
                        self.side1 = BUY
                        self.side2 = SELL
                        price1 = ask1['price']
                        price2 = bid2['price']
                        self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                        self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                        self.open_longs += 1
                        print("open long")
                    elif s > self.short_open:
                        self.side1 = SELL
                        self.side2 = BUY
                        price1 = bid1['price']
                        price2 = ask2['price']
                        self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                        self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                        self.open_shorts += 1
                        print("open short")
                elif pos < 0 and s < self.short_close:
                    self.side1 = BUY
                    self.side2 = SELL
                    price1 = ask1['price']
                    price2 = bid2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                    self.close_shorts += 1
                    print("short close")
                elif pos > 0 and s > -self.long_close:
                    self.side1 = SELL
                    self.side2 = BUY
                    price1 = bid1['price']
                    price2 = ask2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                    self.close_longs += 1
                    print("long close")
            
                    # reset the timer unless we are done 
        if time < self.end_time:
            self.session.add_timer(time + self.interval, self.timer_callback) 
                
            
    def process_executions(self, evp):
        if 'executed_orders' in evp:
            time = self.session.current_time()
            for ex in evp['executed_orders']:
                order = ex['order']
                side = order['side']
                ticker = order['ticker']
                if ticker == self.ticker1:
                    if side == 'B':
                        self.buy_shares1 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares1 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares1 - self.sell_shares1
                elif ticker == self.ticker2:
                    if side == 'B':
                        self.buy_shares2 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares2 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares2 - self.sell_shares2        
                pnl = self.get_pnl()
                print "{0} {1} {quantity_executed} {price_executed} {liquidity} {2} {3}".format(time, side, pos, pnl, **ex)
                

    def get_midmarket(self, ticker):
        bid, ask = self.session.get_inside_market(ticker)
        return (bid['price'] + ask['price']) / 2

    
    def get_pnl(self):
        # mark to mid
        mid1 = self.get_midmarket(self.ticker1)
        mid2 = self.get_midmarket(self.ticker2)
        pnl = self.sell_dollars - self.buy_dollars + (self.buy_shares1 - self.sell_shares1) * mid1 + (self.buy_shares2 - self.sell_shares2) * mid2
        return pnl
    
    def regress(self, returns1,returns2):
        x = np.asarray(returns1).reshape(-1,1)
        y = np.asarray(returns2).reshape(-1,1)
        model = LinearRegression()
        model.fit(x,y)
        a = model.intercept_[0]
        b = model.coef_[0,0]
        residuals = y-model.predict(x)
        return residuals, a,b
    
    def returns(self, midprices):
        return np.diff(midprices, axis=-1)/midprices[:-1]
    
    def fitOU(self, residual):
        ou = np.cumsum(residual)
        model = AR(ou)
        fittedmodel = model.fit(maxlag=1, disp=-1)  
        a = fittedmodel.params[0]
        b = fittedmodel.params[1]
        var =  fittedmodel.sigma2
        kappa = -np.log(b)/self.dt
        m = a/(1-np.exp(-kappa*self.dt))
        sigma = np.sqrt(var*2*kappa/(1-np.exp(-2*kappa*self.dt)))
        sigmaeq = np.sqrt(var/(1-np.exp(-2*kappa*self.dt)));
        return kappa, m, sigma, sigmaeq
    
    def sscore(self, m, sigmaeq):
        if sigmaeq != 0:
            return -m/sigmaeq
        elif m>0:
            return 10000000
        else:
            return -10000000
    
    def end(self):
        print("Total long opens: " + str(self.open_longs))
        print("Total short opens: " + str(self.open_shorts))
        print("Total long closes: " + str(self.close_longs))
        print("Total short closes: " + str(self.close_shorts))
        print('Runs: ' + str(self.runs))
        return self.get_pnl()

date = "20170413"
tickers = ['AAPL', 'MSFT']
start_time = string_to_micro("9:30")
end_time = string_to_micro("16:00")
sim = Simulator(Peggy)
sim.run(date, tickers, use_om=True, start_time=start_time, end_time=end_time)

open short
34303000248 B 1 65080000 R 1 -5000
34303000248 S 1 141870000 R -1 -35000
short close
34344000248 S 1 65220000 R 0 245000
34344000248 B 1 141780000 R 0 230000
open short
34348000249 B 1 65250000 R 1 225000
34348000249 S 1 141780000 R -1 215000
short close
34360000248 S 1 65280000 R 0 245000
34360000248 B 1 141800000 R 0 240000
open long
34413000248 S 1 65270000 R -1 235000
34413000248 B 1 141670000 R 1 230000
long close
34451000248 S 1 141620000 R 0 270000
34463051647 B 1 65190000 A 0 270000
open short
34487000248 B 1 65240000 R 1 265000
34487000248 S 1 141660000 R -1 260000
short close
34510000248 S 1 65190000 R 0 270000
34510000248 B 1 141620000 R 0 260000
open long
34610000248 S 1 65310000 R -1 255000
34610000248 B 1 141820000 R 1 250000
long close
34614000248 B 1 65310000 R 0 220000
34614000248 S 1 141770000 R 0 210000
open long
34617000248 S 1 65290000 R -1 205000
34617000248 B 1 141790000 R 1 195000
long close
34650000248 B 1 65310000 R 0 165000
34650000248 S 1 14176000

36409000248 B 1 142160000 R 0 -750000
open short
36444000248 B 1 65620000 R 1 -755000
36444000248 S 1 142100000 R -1 -760000
short close
36466000248 S 1 65610000 R 0 -770000
36466000248 B 1 142120000 R 0 -780000
open short
36490000248 B 1 65620000 R 1 -785000
36490000248 S 1 142080000 R -1 -790000
short close
36526000248 S 1 65620000 R 0 -805000
36526000248 B 1 142110000 R 0 -810000
open long
36537000248 S 1 65610000 R -1 -815000
36537000248 B 1 142110000 R 1 -820000
long close
36552000248 B 1 65630000 R 0 -815000
36552000248 S 1 142120000 R 0 -820000
open short
36678000248 B 1 65630000 R 1 -825000
36678000248 S 1 142100000 R -1 -830000
short close
36684000248 S 1 65630000 R 0 -845000
36684000248 B 1 142130000 R 0 -850000
open long
36771000248 S 1 65630000 R -1 -855000
36771000248 B 1 142180000 R 1 -860000
long close
36779000248 B 1 65640000 R 0 -865000
36779000248 S 1 142170000 R 0 -870000
open long
36788000248 S 1 65620000 R -1 -875000
36791533647 B 1 142170000 A 1 -880000
long close

long close
38757000248 B 1 65770000 R 0 -1785000
38757000248 S 1 142210000 R 0 -1790000
open long
38769000248 S 1 65750000 R -1 -1795000
38769000248 B 1 142220000 R 1 -1800000
long close
38770000248 B 1 65770000 R 0 -1815000
38770000248 S 1 142210000 R 0 -1820000
open long
38781000248 S 1 65760000 R -1 -1825000
38781000248 B 1 142250000 R 1 -1830000
long close
38791000248 B 1 65760000 R 0 -1855000
38791000248 S 1 142210000 R 0 -1860000
open long
38805000248 S 1 65750000 R -1 -1865000
38805000248 B 1 142240000 R 1 -1870000
long close
38824000248 B 1 65760000 R 0 -1885000
38824000248 S 1 142220000 R 0 -1890000
open long
38867000248 S 1 65740000 R -1 -1895000
38867000248 B 1 142220000 R 1 -1900000
long close
38870000248 B 1 65750000 R 0 -1935000
38870000248 S 1 142180000 R 0 -1940000
open long
38884000248 S 1 65730000 R -1 -1945000
38884000248 B 1 142200000 R 1 -1950000
long close
38912000248 B 1 65730000 R 0 -2025000
38912000248 S 1 142110000 R 0 -2030000
open short
38940000248 B 1 65750

open short
40703000248 B 1 65620000 R 1 -2955000
40703000248 S 1 142160000 R -1 -2960000
short close
40720000248 S 1 65620000 R 0 -2995000
40720000248 B 1 142210000 R 0 -3000000
open long
40784000248 S 1 65610000 R -1 -3005000
40784000248 B 1 142220000 R 1 -3015000
long close
40806000248 B 1 65620000 R 0 -3015000
40806000248 S 1 142210000 R 0 -3020000
open short
40874000248 B 1 65630000 R 1 -3025000
40874000248 S 1 142190000 R -1 -3030000
short close
40913000248 S 1 65640000 R 0 -3065000
40913000248 B 1 142250000 R 0 -3070000
open long
40930000248 S 1 65630000 R -1 -3075000
40930000248 B 1 142230000 R 1 -3080000
long close
40979000248 B 1 65640000 R 0 -3045000
40979000248 S 1 142260000 R 0 -3050000
open short
41013000248 B 1 65640000 R 1 -3055000
41013000248 S 1 142260000 R -1 -3060000
short close
41024000248 S 1 65630000 R 0 -3075000
41024000248 B 1 142280000 R 0 -3080000
open long
41025000248 S 1 65630000 R -1 -3085000
41025000248 B 1 142280000 R 1 -3090000
long close
41029000248 B 1

43308000248 B 1 142150000 R 1 -3850000
long close
43343000248 B 1 65530000 R 0 -3835000
43343000248 S 1 142150000 R 0 -3840000
open short
43404000248 B 1 65510000 R 1 -3845000
43404000248 S 1 142130000 R -1 -3850000
short close
43405000248 S 1 65490000 R 0 -3865000
43405000248 B 1 142140000 R 0 -3870000
open short
43429000248 B 1 65500000 R 1 -3875000
43429000248 S 1 142130000 R -1 -3880000
short close
43431000248 S 1 65500000 R 0 -3885000
43431000248 B 1 142150000 R 0 -3890000
open short
43447000248 B 1 65510000 R 1 -3895000
43447000248 S 1 142150000 R -1 -3900000
short close
43473000248 S 1 65490000 R 0 -3915000
43473000248 B 1 142160000 R 0 -3920000
open long
43556000248 S 1 65430000 R -1 -3925000
43556000248 B 1 142060000 R 1 -3930000
long close
43569000248 B 1 65450000 R 0 -3975000
43569000248 S 1 142020000 R 0 -3980000
open short
43630000248 B 1 65460000 R 1 -3985000
43630000248 S 1 142080000 R -1 -3990000
short close
43661000248 S 1 65460000 R 0 -4005000
43661000248 B 1 14211000

...

52063000248 S 1 141880000 R 0 -8240000
open long
52111000248 S 1 65270000 R -1 -8245000
52111000248 B 1 141890000 R 1 -8250000
long close
52119000248 B 1 65290000 R 0 -8240000
52119157047 S 1 141910000 A 0 -8240000
open short
52142000248 B 1 65290000 R 1 -8245000
52142000248 S 1 141920000 R -1 -8250000
short close
52152000250 S 1 65280000 R 0 -8265000
52152000250 B 1 141940000 R 0 -8270000
open short
52153000248 B 1 65300000 R 1 -8275000
52153000248 S 1 141930000 R -1 -8280000
short close
52154000248 S 1 65290000 R 0 -8285000
52154000248 B 1 141940000 R 0 -8290000
open short
52167000248 B 1 65290000 R 1 -8295000
52167000248 S 1 141890000 R -1 -8300000
short close
52168000248 S 1 65280000 R 0 -8325000
52168000248 B 1 141920000 R 0 -8330000
open long
52175000248 S 1 65270000 R -1 -8335000
52175000248 B 1 141910000 R 1 -8340000
long close
52180000248 B 1 65280000 R 0 -8355000
52180000248 S 1 141890000 R 0 -8360000
open long
52185000248 S 1 65270000 R -1 -8365000
52185000248 B 1 141910000 

53733000248 S 1 65190000 R -1 -9495000
53733000248 B 1 141890000 R 1 -9500000
long close
53745000248 B 1 65210000 R 0 -9485000
53745000248 S 1 141910000 R 0 -9490000
open long
53752000248 S 1 65190000 R -1 -9495000
53752000248 B 1 141920000 R 1 -9500000
long close
53763000248 B 1 65210000 R 0 -9515000
53763000248 S 1 141910000 R 0 -9520000
open short
53850000248 B 1 65230000 R 1 -9525000
53850000248 S 1 141940000 R -1 -9530000
short close
53852000248 S 1 65220000 R 0 -9545000
53852000248 B 1 141960000 R 0 -9550000
open short
53876000248 B 1 65240000 R 1 -9555000
53876000248 S 1 141960000 R -1 -9560000
short close
53896000248 S 1 65230000 R 0 -9545000
53896000248 B 1 141950000 R 0 -9550000
open short
53912000248 B 1 65250000 R 1 -9555000
53912000248 S 1 141940000 R -1 -9560000
short close
53920000248 S 1 65240000 R 0 -9575000
53920000248 B 1 141960000 R 0 -9580000
open short
53921000248 B 1 65260000 R 1 -9585000
53921000248 S 1 141950000 R -1 -9590000
short close
53926000248 S 1 6524000

open short
55481000248 B 1 65250000 R 1 -10595000
55481000248 S 1 141840000 R -1 -10600000
short close
55482000248 S 1 65240000 R 0 -10605000
55482000248 B 1 141850000 R 0 -10610000
open short
55483000248 B 1 65250000 R 1 -10615000
55483000248 S 1 141840000 R -1 -10620000
short close
55484000248 S 1 65240000 R 0 -10625000
55484000248 B 1 141850000 R 0 -10630000
open long
55488000248 S 1 65230000 R -1 -10635000
55488000248 B 1 141850000 R 1 -10640000
long close
55516000248 B 1 65240000 R 0 -10655000
55516000248 S 1 141830000 R 0 -10660000
open long
55610000248 S 1 65210000 R -1 -10665000
55610000248 B 1 141800000 R 1 -10670000
long close
55611000248 B 1 65230000 R 0 -10675000
55611000248 S 1 141800000 R 0 -10680000
open short
55625000248 B 1 65230000 R 1 -10685000
55625000248 S 1 141810000 R -1 -10695000
short close
55628000248 S 1 65220000 R 0 -10705000
55628000248 B 1 141830000 R 0 -10710000
open short
55629000248 B 1 65240000 R 1 -10715000
55629000248 S 1 141820000 R -1 -10720000
sho

57288000248 S 1 141400000 R -1 -11895000
short close
57293000248 S 1 65090000 R 0 -11875000
57293000248 B 1 141370000 R 0 -11880000
open short
57303000248 B 1 65120000 R 1 -11885000
57303007247 S 1 141390000 A -1 -11890000
short close
57306000248 S 1 65100000 R 0 -11895000
57306000248 B 1 141390000 R 0 -11900000
open long
57346000248 S 1 65100000 R -1 -11905000
57346000248 B 1 141420000 R 1 -11910000
long close
57384000248 B 1 65100000 R 0 -11915000
57384000248 S 1 141400000 R 0 -11920000
open short
57423000248 B 1 65090000 R 1 -11925000
57423000248 S 1 141390000 R -1 -11930000
short close
57425000248 S 1 65070000 R 0 -11925000
57425000248 B 1 141380000 R 0 -11930000
open short
57484000248 B 1 65070000 R 1 -11935000
57484000248 S 1 141310000 R -1 -11940000
short close
57485000248 S 1 65040000 R 0 -11945000
57485000248 B 1 141300000 R 0 -11950000
open short
57492000248 B 1 65050000 R 1 -11955000
57492000248 S 1 141250000 R -1 -11960000
short close
57496000248 S 1 65030000 R 0 -11975000


-12060000